In [14]:
# Import libraries and packages
import pandas as pd
import numpy as np
import os
import math
import tableone
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [15]:
screening_method = 'inf_phoenix_septic_shock'

In [16]:
path = '/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening'
comp_cohort = pd.read_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/complete_cohort.csv')
inf_phoenix_eg = pd.read_parquet(os.path.join('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models', 'dataset_agg_eg_inf_phoenix.parquet.gzip'))
inf_phoenix_sr = pd.read_parquet(os.path.join('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_models', 'dataset_agg_sr_inf_phoenix.parquet.gzip'))
inf_phoenix = pd.concat([inf_phoenix_eg, inf_phoenix_sr])
comp_cohort = comp_cohort[comp_cohort['csn'].isin(inf_phoenix['csn'].unique().tolist())]
data = comp_cohort[['patid', 'csn', 'dob']]
data['dob'] = data['dob'].apply(pd.to_datetime)

In [17]:
# Load data
cohort_pos = pd.read_csv(os.path.join('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening', 'cohort_' + screening_method + '.csv'))

# Add sepsis label
data['Sepsis'] = 0
data.loc[data['csn'].isin(cohort_pos['csn'].unique().tolist()), 'Sepsis'] = 1

# Load demographics file
demo = pd.read_parquet("/labs/kamaleswaranlab/ECMO/new_data/TAB1_Patients.parquet.gzip")
demo = demo[['Pat ID', 'Gender', 'Race']]
demo.columns = ['patid', 'Gender', 'Race']

# Add race and gender
data = pd.merge(data, demo, on='patid', how="left")

# Fix gender
data['Gender'] = data['Gender'].fillna('Unknown')

# Fix race
data['Race'] = data['Race'].fillna('Unknown')
data.loc[(data['Race'].str.contains(";", case=False)) & (data['Race'].str.contains("declined|unknown", case=False)), 'Race'] = 'Unknown'
data.loc[data['Race'].str.contains(";", case=False), 'Race'] = 'Other'
data.loc[data['Race'].isin(['Black/African-Amer']), 'Race'] = 'Black or African American'
data.loc[data['Race'].isin([' White,Non-Hipanic', 'White,Hispanic']), 'Race'] = 'White'
data.loc[data['Race'].isin(['Declined', 'Non-White Hispanic', 'Other', 'Other/Declined', 'Patient Not Present', 'Parent Not Present']), 'Race'] = 'Unknown'
data.loc[data['Race'].isin(['American Ind/Alaskan', 'Multi-Racial', 'American Indian or Alaska Native', 'Native Hawaiian or Other Pacific Islander']), 'Race'] = 'Other'

In [19]:
# Load encounters file
print('Loading encounters...')
dept_path = '/labs/kamaleswaranlab/ECMO/new_data/TAB2_Encounter_Departments.parquet.gzip'
dept = pd.read_parquet(dept_path)
dept[['BIRTH_DATE', 'Hosp_Admission', 'Hosp_Discharge', 'Entered_Dept', 'Exited_Dept']] = dept[['BIRTH_DATE', 'Hosp_Admission', 'Hosp_Discharge', 'Entered_Dept', 'Exited_Dept']].apply(pd.to_datetime)
dept = dept[['Pat ID', 'Encounter CSN', 'Name', 'BIRTH_DATE', 'Department', 'Entered_Dept', 'Exited_Dept', 'Hosp_Admission', 'Hosp_Discharge']]
dept.columns = ['patid', 'csn', 'name', 'dob', 'department', 'entered_dept', 'exited_dept', 'hosp_adm', 'hosp_disch']
dept = dept[(dept['department'].str.contains('PEDIATRIC ICU')) & (dept['hosp_adm'] >= '2010-01-01')]
dept.dropna(inplace=True)
dept.drop_duplicates(inplace=True)

dept_first = dept.sort_values(by=['csn', 'hosp_adm', 'entered_dept'])
dept_first = dept_first.groupby('csn', as_index=False).first()

# Add hospital and PICU admission and discharge
data = data.merge(dept_first[['csn', 'department', 'entered_dept', 'exited_dept', 'hosp_adm', 'hosp_disch']], how='inner', on='csn')

Loading encounters...


In [20]:
# Add age
data['Age in years'] = round(round((data['hosp_adm'] - data['dob']) / pd.Timedelta('1 day'), 0) / 365.25, 2)

# Compute Age group
data['Age Group'] = np.where(data['Age in years'] <= 0.083, '≤ 28 days', np.where(data['Age in years'] <= 3.0, '29 days - 2 years', 
                    np.where(data['Age in years'] <= 6.0, '3 - 5 years', '6 - 17 years')))

In [21]:
# Compute hospital LOS
data['Hospital Length of Stay in days'] = (data['hosp_disch'] - data['hosp_adm']) / pd.Timedelta('1 day')

# Compute PICU LOS
data['PICU Length of Stay in days'] = (data['exited_dept'] - data['entered_dept']) / pd.Timedelta('1 day')
data.drop(['hosp_disch', 'hosp_adm', 'exited_dept'], axis=1, inplace=True)

# Add mortality
depts = pd.read_parquet(dept_path)
depts[['Entered_Dept']] = depts[['Entered_Dept']].apply(pd.to_datetime)
depts['Hospital mortality'] = np.where(depts['Hospital_Discharge_Disposition'] == 'Expired', 'Yes', np.where(depts['Hospital_Discharge_Disposition'] == 'Expired Place Unknown', 'Yes', 'No'))
depts = depts[['Encounter CSN', 'Hospital mortality', 'Department', 'Entered_Dept']]
depts.columns = ['csn', 'Hospital Mortality', 'department', 'entered_dept']

# Merge dataframes
data = data.merge(depts, how='inner', on=['csn', 'department', 'entered_dept'])
data.rename(columns={'department':'PICU Campus'}, inplace=True)

# Add ethnicity
pats = pd.read_parquet("/labs/kamaleswaranlab/ECMO/new_data/TAB1_Patients.parquet.gzip")
pats = pats[['Pat ID', 'Ethnicity']]
pats.columns = ['patid', 'Ethnicity']

# Merge dataframes
data = data.merge(pats, how='inner', on='patid')
data.drop(['entered_dept'], axis=1, inplace=True)

In [22]:
# Add mortality scores
scores = pd.read_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_analysis/scores_24.csv')
scores = scores[['csn', 'phoenix', 'psofa', 'pelod2', 'prism3']]
scores.columns = ['csn', 'Phoenix', 'pSOFA', 'PELOD II', 'PRISM III']
data = data.merge(scores, how='left', on='csn')

In [23]:
# Organize data
data.loc[(data['PICU Campus'] == 'SR PEDIATRIC ICU') & ((data['Sepsis'] == 0)), 'Campus'] = 'Validation No Sepsis'
data.loc[(data['PICU Campus'] == 'EG PEDIATRIC ICU') & ((data['Sepsis'] == 0)), 'Campus'] = 'Derivation No Sepsis'
data.loc[(data['PICU Campus'] == 'SR PEDIATRIC ICU') & ((data['Sepsis'] == 1)), 'Campus'] = 'Validation Sepsis'
data.loc[(data['PICU Campus'] == 'EG PEDIATRIC ICU') & ((data['Sepsis'] == 1)), 'Campus'] = 'Derivation Sepsis'

data.drop(['Sepsis', 'PICU Campus', 'patid', 'csn', 'dob'], axis=1, inplace=True)

# Fix Ethnicity
data.loc[data['Ethnicity'].isin(['Declined', 'Patient Not Present', 'Parent Not Present']), 'Ethnicity'] = 'Unknown'

In [24]:
# Create tableOne
columns = list(data.columns)
columns.remove('Campus')

categorical = ['Gender', 'Race', 'Age Group', 'Hospital Mortality', 'Ethnicity']
    
nonnormal = [x for x in columns if x not in categorical]

groupby = ['Campus']

data.reset_index(inplace=True, drop=True)

myTable = tableone.TableOne(data, columns=columns, categorical=categorical, groupby=groupby, nonnormal=nonnormal, pval=False, missing=False, htest_name=False)
myTable

Grouped by Campus                                                                              
                                                                                    Overall Derivation No Sepsis Derivation Sepsis Validation No Sepsis Validation Sepsis
n                                                                                     63875                27178              1140                34354              1203
Gender, n (%)                                   Female                         28774 (45.0)         12285 (45.2)        513 (45.0)         15399 (44.8)        577 (48.0)
                                                Male                           35098 (54.9)         14891 (54.8)        627 (55.0)         18954 (55.2)        626 (52.0)
                                                Unknown                             3 (0.0)              2 (0.0)                                1 (0.0)                  
Race, n (%)                                     Asian                            1914 (3.0)            654 (2.4)          36 (3.2)           1163 (3.4)          61 (5.1)
                                                Black or African American      25828 (40.4)         14263 (52.5)        575 (50.4)         10609 (30.9)        381 (31.7)
                                                Other                             279 (0.4)            116 (0.4)           3 (0.3)            157 (0.5)           3 (0.2)
                                                Unknown                         9056 (14.2)           2666 (9.8)         109 (9.6)          6088 (17.7)        193 (16.0)
                                                White                          26798 (42.0)          9479 (34.9)        417 (36.6)         16337 (47.6)        565 (47.0)
Age in years, median [Q1,Q3]                                                 4.4 [1.1,11.6]       4.7 [1.2,11.9]    6.7 [1.7,13.1]       3.9 [1.0,11.1]    8.0 [2.5,13.4]
Age Group, n (%)                                29 days - 2 years              25222 (39.5)         10595 (39.0)        326 (28.6)         14020 (40.8)        281 (23.4)
                                                3 - 5 years                     8881 (13.9)          3669 (13.5)        153 (13.4)          4898 (14.3)        161 (13.4)
                                                6 - 17 years                   27574 (43.2)         12169 (44.8)        607 (53.2)         14086 (41.0)        712 (59.2)
                                                ≤ 28 days                        2198 (3.4)            745 (2.7)          54 (4.7)           1350 (3.9)          49 (4.1)
Hospital Length of Stay in days, median [Q1,Q3]                               3.9 [2.2,8.0]        3.7 [2.0,8.1]   11.6 [5.2,21.5]        3.9 [2.4,7.3]    9.0 [4.7,20.9]
PICU Length of Stay in days, median [Q1,Q3]                                   1.4 [0.7,2.8]        1.3 [0.6,2.6]     2.3 [0.1,6.8]        1.5 [0.8,2.8]     2.7 [0.6,6.8]
Hospital Mortality, n (%)                       No                             62384 (97.7)         26533 (97.6)        904 (79.3)         33894 (98.7)       1053 (87.5)
                                                Yes                              1491 (2.3)            645 (2.4)        236 (20.7)            460 (1.3)        150 (12.5)
Ethnicity, n (%)                                Hispanic or Latino              8918 (14.5)          2637 (10.1)        143 (12.5)          5874 (17.7)        264 (21.9)
                                                Non Hispanic or Latino         52300 (84.9)         23177 (89.1)        996 (87.4)         27190 (81.8)        937 (77.9)
                                                Unknown                           393 (0.6)            201 (0.8)           1 (0.1)            189 (0.6)           2 (0.2)
Phoenix, median [Q1,Q3]                                                       0.0 [0.0,1.0]        0.0 [0.0,1.0]     3.0 [2.0,5.0]        0.0 [0.0,1.0]     3.0 [2.0

In [55]:
myTable.to_csv(os.path.join('/home/dchanci/research/pediatric_sepsis/prediction_ml/models/results_updated', screening_method, 'tableone.csv'))